# Finding Your Way Around Spark

This lets you try out how to use Spark in the raspberry pi cluster.

##  Taking Spark for a First Spin
First of all, let's see if everything works as it should. Run this code in rpi0, what you are actually using here is the Python shell (pyspark-shell)


In [ ]:
pyspark --master spark://rpi0:7077

Let's try something a bit more sophisticated now. First we create a range of numbers in the form of a Spark DataFrame. This range of numbers is just like a named column in a spreadsheet, with the big difference that it can be distributed.

using a so-called transformation on a data set. We'll use the same data set as above, but we will apply a filter to it, getting rid of the even values..

In [ ]:
myRange = spark.range(1000).toDF("number")

In [ ]:
divBy2 = myRange.where("number % 2 = 0")

In order to trigger some activity, we are now going to apply an action. Running this code will execute the transformation and then count the records. Unsurprisingly, the result is 500.

In [ ]:
divBy2.count()

## A First Application

What we have done so far is not too exciting. We will now look at a more elaborate example, seeing some more functionality of Spark. First of all, we have to upload a file called 2015-summary.csv containing some flight data onto the hdfs file system. 

In [ ]:
hadoop fs -copyFromLocal 2015-summary.csv /data

Spark includes the ability to read and write from a large number of data sources. In order to read this data, we use a DataFrameReader that is associated with our SparkSession. We also specify the file format as well as some other options. We want Spark to take a best guess at what the schema of our DataFrame should be and interpret the first row of the file as the header. To check whether things have worked out, we look at the first three rows.

In [ ]:
flightData2015 = spark.read.option("inferSchema", "true").option("header", "true").csv("hdfs://rpi0:8020/data/2015-summary.csv")
flightData2015.take(3)

Next, let's have a look at the schema of the DataFrame storing the flight data. The schema is fairly simple: a destination country, a country of origin, and the number of flights between these two countries in 2015.

For a quick overview on how to handle DataFrames in Spark, you can have a look here:
https://spark.apache.org/docs/3.4.0/api/python/getting_started/quickstart_df.html

In [ ]:
flightData2015.printSchema()

After uploading the file and checking the first 10 tuples with "head", let's now do the same using Spark, i.e., look at the first 10 tuples. You should see the same information as above.

In [ ]:
flightData2015.show(10)

Let's do some more transformations! We sort the data according to the count column, which is an integer. Remember that sort does not actually modify the DataFrame. It is a transformation that returns a new DataFrame. Nothing happens until we apply an action, so let's take the first three rows after sorting.

In [ ]:
flightData2015.sort("count").take(3)

We can also have a look at the plan built by Spark to execute the sort. This is done by calling the function explain, which will show us the lineage graph. This allows Spark to recompute any partition by performing all of the  operations on the input data. By passing the parameter true (or 1 in Python) to explain (calling explain(1) instead of explain()), we get additional information on the optimization process.

In [ ]:
flightData2015.sort("count").explain()

It is also possible to submit SQL queries to a DataFrame. First we have to turn the DataFrame into a table or view. The triple quotation marks (" " ") allow the SQL statement to span multiple lines.

In [ ]:
flightData2015.createOrReplaceTempView("flight_data_2015")
sqlWay = spark.sql("""
select dest_country_name, count(1)
from flight_data_2015
group by dest_country_name
""")
sqlWay.take(3)

We could also do this via Spark directly. The underlying plan would be the same in both cases.

In [ ]:
dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

sqlWay.explain()
dataFrameWay.explain()

As a small exercise, now group the data according to the country of origins and count the number of flights from every country. Output the top three countries, i.e., the three countries with the most outbound flights. Hint: if you want to sort in descending order, you need an additional parameter: ascending=False, e.g.

flightData2015.sort("count", ascending=False)

## Wordcount in Map Reduce

We now want to implement a Map Reduce process in Spark that counts words for us. First of all, we have a look at how to manipulate text files, so we can get them into the right form for Map Reduce. First we create the file we are using for wordcount (you can also upload a file)


In [ ]:
%%writefile file.txt
touch file.txt
nano file.txt

pease porridge hot pease porridge cold
pease porridge in the pot
nine days old
in the pot cold in the pot hot
pease porridge pease porridge
eat the lot

hadoop fs -copyFromLocal file.txt /data

Then we run the actual wordcount scheme on this file. The operators we call (flatMap, map, and reduceByKey) need a function as an input parameter to tell them what to do to the data. We can create anonymous functions on the fly using so-called lambda function. A lambda function can take any number of arguments, but can only have one expression.

In [ ]:
x = lambda a : a + 10
print(x(5))

Now we are ready to run the actual wordcount scheme on the text file.

In [ ]:
text = sc.textFile("hdfs://rpi0:8020/data/file.txt")
textsplit = text.flatMap(lambda line: line.split(" "))
tuples = textsplit.map(lambda word: (word,1))
counts = tuples.reduceByKey(lambda a, b: a + b)

for x in counts.collect():
    print(x)

Change some of the lines in file.txt to see what happens with the frequencies of the words when you run wordcount again.

## Building an inverted index
Now try to build an inverted index for the text file above. Below we create a subdirectory and then split up the lines into separate documents. You can also upload other files into the directory 'files' to create an inverted index for another document collection.

In [ ]:
mkdir files

%%writefile files/file1.txt
touch files/file1.txt
nano files/file1.txt
pease porridge hot pease porridge cold

%%writefile files/file2.txt
touch files/file2.txt
nano files/file2.txt
pease porridge in the pot

%%writefile files/file3.txt
touch files/file3.txt
nano files/file3.txt
nine days old

%%writefile files/file4.txt
touch files/file4.txt
nano files/file4.txt
in the pot cold in the pot hot

%%writefile files/file5.txt
touch files/file5.txt
nano files/file5.txt
pease porridge pease porridge

%%writefile files/file6.txt
touch files/file6.txt
nano files/file6.txt
eat the lot

hadoop fs -copyFromLocal files /data

Now we are ready to start the actual work. We can read in all the files of a subdirectory with the function wholeTextFiles. In the next step we split up the content of the document and tag it with the filename, so docsplit will contain tuples of the form (filename, word 1), (filename, word2), etc. In the lambda function used as a parameter for the flatMap operator we use a so-called list comprehension. Let's first have a look at list comprehensions. Assume we want to select the even numbers in a range. We could do this in a for-loop. However, a list comprehension does all this in a single statement.

In [ ]:
even = []
r = range(20)
for i in r:
  if i % 2 == 0:
    even.append(i)
print(even)

evenlc = [x for x in range(20) if x % 2 == 0]
print(evenlc)

Let's now read the files and do the preprocesing.

In [ ]:
documents = sc.wholeTextFiles("hdfs://rpi0:8020/data/files")
docsplit = documents.flatMap(lambda tuple: [(tuple[0],word) for word in tuple[1].split(" ")])

for i in docsplit.collect():
  print(i)

Now write the actual map and reduce steps. Feel free to play around with the code to try out different things.

# An Introduction to Spark Streaming

In the second half of this exercise we give a brief introduction to stream processing in Apache Spark, running an aggregation and a transformation query over the data.

## A Streaming Application

The file contains some event data and has the following schema:

root \\
 |-- Arrival_Time: long (nullable = true) \\
 |-- Creation_Time: long (nullable = true) \\
 |-- Device: string (nullable = true) \\
 |-- Index: long (nullable = true) \\
 |-- Model: string (nullable = true) \\
 |-- User: string (nullable = true) \\
 |-- gt: string (nullable = true) \\
 |-- x: double (nullable = true) \\
 |-- y: double (nullable = true) \\
 |-- z: double (nullable = true) \\

 The full data set comprises 1.2GByte of data, the file supplied with this exercise is only a small subset. The full dataset can be downloaded here (in the subdirectory data/activity-data): https://github.com/databricks/Spark-The-Definitive-Guide

In [ ]:
tar -zxvf data.tar.gz
hadoop fs -copyFromLocal data/activity-data /data

### Basics

Now that we have uploaded the data, we can start with the actual application. As this is intended for a small machine running in local mode, we are going to set the number of partitions to small number. We first initialize a local spark session:

In [ ]:
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffe.partitions",5)

Next, we are going to read the data and store it in a DataFrame. The output of the printSchema operator should look like the schema shown above. We are also going to store the schema in a variable for later use.

In [ ]:
static = spark.read.json("hdfs://rpi0:8020/data/activity-data/")
static.printSchema()
dataSchema = static.schema

### An Aggregation Query

Now it is time to create a streaming version of the DataFrame. The option maxFilesPerTrigger essentially allows us to control how quickly Spark
will read all of the files in the folder. By specifying a small value, we are artificially limiting the flow of the stream to one
file per trigger. This helps us demonstrate how Structured Streaming runs incrementally in our example, but is not something we would use in production.

In [ ]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).json("hdfs://rpi0:8020/data/activity-data")

So far, Spark has not actually executed anything (Spark follows a lazy execution paradigm). We are going to define a query on the stream and then start the actual processing. The field gt specifies what activity the user was doing at that time. For the actual stream processing, we use memory as output parameter, which means that the result is not written into a file. The mode complete means each output will be a complete result (and not just be appended to a previous result). We then check which streams are currently active.

In [ ]:
activityCounts = streaming.groupBy("gt").count()
activityQuery = activityCounts.writeStream.queryName("activity_counts").format("memory").outputMode("complete").start()
spark.streams.active

If we want to look at the actual results, we have to look at the content of the results of the query activity_counts. If we call this multiple times, the results should change. Depending on the machine you are running this on, as we are using a very small data set, we might not be able to see this. However, with a larger dataset, the result will be updated from time to time.

In [ ]:
spark.streams.active

In [ ]:
spark.sql("select * from activity_counts").show()

If we want to rerun the query on the stream, we first have to stop the current version. Then we can rerun the activityQuery shown above again.

In [ ]:
activityQuery.stop()
spark.streams.active

### A Transformation Query

Next, we are going to run a transformation query that filters out all the events where gt is equal to bike and the device that was used is a Nexus 4.2. Again, we keep the result in main memory, but this time we append the newly created data to the previous result. The number of events should go up over time.

In [ ]:
simpleTransform = streaming.where("gt = 'bike'").where("Device ='nexus4_2'")
transformQuery = simpleTransform.writeStream.queryName("simple_transform").format("memory").outputMode("append").start()
spark.streams.active

As before, depending on the machine and the size of the dataset, this might be a bit difficult to see. Just try out the two queries below and call the one counting the events multiple times.

In [ ]:
spark.sql("select * from simple_transform").show()

In [ ]:
spark.sql("select count(*) from simple_transform").show()

After we are done, we have to stop the stream processing again. Otherwise it will keep running indefinitely, waiting for new input data to process.

In [ ]:
transformQuery.stop()
spark.streams.active

### Windowing

In the final part of this exercise, we are going to run some queries on windows: tumbling windows and sliding windows. Before we can do so, we have to convert the timestamps, though. Spark SQL uses a different data type. We append this an additional field called event_time.

In [ ]:
withEventTime = streaming.selectExpr("*", "cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time")
withEventTime.printSchema()

We can now start processing the data. The simplest operation is simply to count the number of occurrences of an event in a given window by performing an aggregation of the keys over a window of time. We use non-overlapping windows (tumbling windows) of 10 minutes. First we import some functionality we need:


In [ ]:
from pyspark.sql.functions import window, col

The following commands create a query over windows of length 10 minutes that group the data using the field event_time.

In [ ]:
winCounts = withEventTime.groupBy(window(col("event_time"), "10 minutes")).count()
winQuery = winCounts.writeStream.queryName("events_per_win").format("memory").outputMode("complete").start()
spark.streams.active

In order to see the result, we need to query events_per_win. We just look at the first 20 events. The paramter False specifies that we want to see the whole content of a field (otherwise it gets cropped).

In [ ]:
spark.sql("select * from events_per_win order by window").show(20,False)

Finally, we stop the stream processing again.

In [ ]:
winQuery.stop()
spark.streams.active

**Small Exercise:** Take the windowing query from above and increse the window size to 40 minutes. Compare the output to the one with a window size of 10 minutes. 

Now formulate the query and run it on the stream.

Do not forget to stop the stream processing once you are done with the exercise.

Now we do the same thing, but use sliding windows of 10 minutes. However, when moving to the next window, we only shift by 5 minutes. So, all neighboring windows overlap each other by 5 minutes.

In [ ]:
winCounts2 = withEventTime.groupBy(window(col("event_time"), "10 minutes", "5 minutes")).count()
winQuery2 = winCounts2.writeStream.queryName("events_per_win").format("memory").outputMode("complete").start()
spark.streams.active

When you compare the output of the following query to the output of the query above, you can see that the windows are now overlapping.

In [ ]:
spark.sql("select * from events_per_win order by window").show(20,False)

In [ ]:
winQuery2.stop()
spark.streams.active

The way it is specified above, the windows would be kept around indefinitely, waiting for late-arriving data. This is not feasible: at some point we would run out of memory if data keeps arriving in a stream. For that reason, we specify a watermark, which tells the system how long to keep a window open for updates. Any data arriving later than that will not be considered anymore. Which duration to choose depends on the application. Here we assume that there can be some delay (if a user loses mobile phone connectivity) and decide to wait for up to 30 minutes.


In [ ]:
winCounts3 = withEventTime.withWatermark("event_time", "30 minutes").groupBy(window(col("event_time"), "10 minutes", "5 minutes")).count()
winQuery3 = winCounts3.writeStream.queryName("events_per_win").format("memory").outputMode("complete").start()
spark.streams.active

When querying the table events_per_win we can see intermediate results, because we are using the output mode "complete". In append mode, the information would not be available until a window closes.

In [ ]:
spark.sql("select * from events_per_win order by window").show(20,False)

In [ ]:
winQuery3.stop()
spark.streams.active

Finally, we look at getting rid of duplicates in a stream. When messages are generated in non-stable network environments, such as Internet-of-Things (IoT) networks, messages may be sent out multiple times. In order to drop duplicates, we first need to specify what constitutes a duplicate for us. In our case, it is an event from the same User at exactly the same event time. This time we group the events by user.

In [ ]:
winCounts4 = withEventTime.withWatermark("event_time", "30 minutes").dropDuplicates(["User", "event_time"]).groupBy("User").count()
winQuery4 = winCounts4.writeStream.queryName("events_per_win").format("memory").outputMode("complete").start()
spark.streams.active

In [ ]:
spark.sql("select * from events_per_win order by User").show()

In [ ]:
winQuery4.stop()
spark.streams.active

During the course of the exercise you got a first glimpse of the power of Spark both in the offline processing and in the streaming context. 